In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report

In [2]:
filetrain=pd.read_csv("claimsss.csv")
col1=filetrain['claim']
col2=filetrain['label']
for i in range(2):
    print(col1[i]," ",col2[i])

Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.   SUPPORTS
Roman Atwood is a content creator.   SUPPORTS


In [3]:
from bs4 import BeautifulSoup

text1=[]
for i in range(len(col1)):
    soup = BeautifulSoup(col1[i])
    text = soup.get_text()
    text1.append(text)         #Removed tags and urls
    
# for i in range(2):
#     print(text1[i])

In [4]:
import nltk
nltk.download('punkt') # For tokenizers
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
words=[]
sum=0
stop_words=set(stopwords.words('english'))
for i in range(len(text1)):
    words.append(nltk.word_tokenize(text1[i]))
    words[i]=[word.lower() for word in words[i] if word.isalnum()]   #removed non alphanumeric
    words[i]=[word.lower() for word in words[i] if word not in stop_words]  #removed stopwords
    sum=sum+len(words[i])
    # print(len(words[i]))


In [6]:
sum2=0
words2=words
words3=[]
Max=0
lemmatizer = WordNetLemmatizer()
for i in range(len(words2)):
    tempstr=""
    for wd in range(len(words2[i])):
        words2[i][wd]=lemmatizer.lemmatize(words2[i][wd]) 
        tempstr=tempstr+' '+words2[i][wd]
    words3.append(tempstr)
    # words2[i]=set(words2[i])
    sum2=sum2+(len(words2[i]))       #lemmatization completed
    if(len(words2[i])>Max):
        Max=len(words2[i])



In [7]:
# Print Statistics of Data like avg length of sentence , proposition of data w.r.t class labels
# print(len(col1))
count=0
cnt=0
for i in col2:
    if(i=='SUPPORTS'):
        count=count+1
    if(i=='REFUTES'):
        cnt=cnt+1
print("Positive reviews = ",count,"\nNegative reviews = ",cnt)
print("average no. of words per review = ",sum/len(col2))
print("maximum number of words = ",Max)
# print("average no. of words after lemmatization = ",sum2/len(col2))
# print(col2.count("positive"))



Positive reviews =  80035 
Negative reviews =  29775
average no. of words per review =  5.175922047172389
maximum number of words =  56


In [9]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = '<OOK>'
embedding_dim = 100
max_length = 200# choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'

In [11]:
reviews = words3
labels = col2

In [12]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)
print((encoded_labels))
print(encoder.classes_)

[1 1 1 ... 1 1 1]
['REFUTES' 'SUPPORTS']


In [13]:
train_sentences, test_sentences, train_labels, test_labels=train_test_split(reviews,encoded_labels,test_size=0.2)

In [14]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [15]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 100)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 24)                3096      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [16]:
num_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
2471/2471 [==============================] - 277s 111ms/step - loss: 0.5096 - accuracy: 0.7728 - val_loss: 0.4913 - val_accuracy: 0.7787
Epoch 2/5
2471/2471 [==============================] - 414s 168ms/step - loss: 0.4707 - accuracy: 0.7878 - val_loss: 0.4833 - val_accuracy: 0.7809
Epoch 3/5
2471/2471 [==============================] - 420s 170ms/step - loss: 0.4486 - accuracy: 0.7957 - val_loss: 0.4830 - val_accuracy: 0.7836
Epoch 4/5
2471/2471 [==============================] - 336s 136ms/step - loss: 0.4309 - accuracy: 0.8005 - val_loss: 0.4877 - val_accuracy: 0.7834
Epoch 5/5
2471/2471 [==============================] - 365s 148ms/step - loss: 0.4154 - accuracy: 0.8067 - val_loss: 0.5022 - val_accuracy: 0.7779


In [17]:
# Calculate accuracy on Test data

prediction = model.predict(test_padded)


# Get probabilities
prob=[]
for i in range(len(prediction)):
    val=prediction[i][0]
    prob.append(val)

# Get labels based on probability 1 if p>= 0.5 else 0
lstmlabels=[]
for i in range(len(prob)):
    if(prob[i]>=0.5):
        ans=1
    else:
        ans=0
    lstmlabels.append(ans)

# Accuracy : one can use classification_report from sklearn
target_names=['negative','positive']
print(classification_report(test_labels,lstmlabels,target_names=target_names))


              precision    recall  f1-score   support

    negative       0.71      0.32      0.44      5969
    positive       0.79      0.95      0.86     15993

    accuracy                           0.78     21962
   macro avg       0.75      0.64      0.65     21962
weighted avg       0.77      0.78      0.75     21962



In [39]:
# reviews on which we need to predict
sentence = ["The collections of the Philadelphia Museum of Art contain over 240,000 objects.", 
            "Daksh Varshney batchie ka kaat ta hai.", 
            "Massachusetts is far away from Rhode Island."]

# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get probabilities
print("Probabilities\n")
randomprob=model.predict(padded)
print(randomprob)
randomlabels=[]

# Get labels based on probability 1 if p>= 0.5 else 0
for i in range(3):
    if(randomprob[i][0]>=0.75):
        randomlabels.append(1)
    else:
        randomlabels.append(0)
for i in range(len(randomlabels)):
    if(randomlabels[i]==1):
        print('positive')
    else:
        print('negative')


Probabilities

[[0.9574223 ]
 [0.81979936]
 [0.5764051 ]]
positive
positive
negative
